In [2]:
import ROOT
import root_numpy as rnpy
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn

In [4]:
ROOT.TGraph(100)

<ROOT.TGraph object ("Graph") at 0x7e7e9d0>